# Face detection and recognition training pipeline

The following example illustrates how to fine-tune an InceptionResnetV1 model on your own dataset. This will mostly follow standard pytorch training patterns.

In [61]:
%pip install facenet_pytorch

Note: you may need to restart the kernel to use updated packages.


In [62]:
from facenet_pytorch import MTCNN, InceptionResnetV1, fixed_image_standardization, training
import torch
from torch.utils.data import DataLoader, SubsetRandomSampler
from torch import optim
from torch.optim.lr_scheduler import MultiStepLR
from torch.utils.tensorboard import SummaryWriter
from torchvision import datasets, transforms
import numpy as np
import os

#### Define run parameters

The dataset should follow the VGGFace2/ImageNet-style directory layout. Modify `data_dir` to the location of the dataset on wish to finetune on.

In [63]:
data_dir = 'C:\\Users\\tyler\\OneDrive\\Documents\\FaceAugment\\images'

batch_size = 32
epochs = 8
workers = 0 if os.name == 'nt' else 4 # 8

#### Determine if an nvidia GPU is available

In [64]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print('Running on device: {}'.format(device))

Running on device: cpu


#### Define MTCNN module

See `help(MTCNN)` for more details.

In [66]:
mtcnn = MTCNN(
    image_size=160, margin=0, min_face_size=20,
    thresholds=[0.6, 0.7, 0.7], factor=0.709, post_process=True,
    device=device
)

#### Perfom MTCNN facial detection

Iterate through the DataLoader object and obtain cropped faces.

In [57]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [67]:
dataset = datasets.ImageFolder(data_dir, transform=transforms.Resize((512, 512)))
dataset.idx_to_class = {i:c for c, i in dataset.class_to_idx.items()}
dataset.samples = [
    (p, p.replace(data_dir, data_dir + '_cropped'))
        for p, _ in dataset.samples
]

# loader = DataLoader(
#     dataset,
#     num_workers=workers,
#     batch_size=batch_size,
#     collate_fn=training.collate_pil
# )
loader = DataLoader(dataset, collate_fn=training.collate_pil, num_workers=workers)

# x: image object
# y: save path of corresponding object
for i, (x, y) in enumerate(loader):
    # print("Save path: ", y[-1])
    mtcnn(x, save_path=y)
    print('\rBatch {} of {}'.format(i + 1, len(loader)), end='')
    
# Remove mtcnn to reduce GPU memory usage
del mtcnn

Batch 124 of 124

#### Define Inception Resnet V1 module

See `help(InceptionResnetV1)` for more details.

In [68]:
resnet = InceptionResnetV1(
    classify=True,
    pretrained='vggface2',
    num_classes=len(dataset.class_to_idx)
).to(device)

100%|██████████| 107M/107M [00:04<00:00, 26.2MB/s] 


#### Define optimizer, scheduler, dataset, and dataloader

In [69]:
optimizer = optim.Adam(resnet.parameters(), lr=0.001)
scheduler = MultiStepLR(optimizer, [5, 10])

trans = transforms.Compose([
    np.float32,
    transforms.ToTensor(),
    fixed_image_standardization
])
dataset = datasets.ImageFolder(data_dir + '_cropped', transform=trans)
img_inds = np.arange(len(dataset))
np.random.shuffle(img_inds)
train_inds = img_inds[:int(0.8 * len(img_inds))]
val_inds = img_inds[int(0.8 * len(img_inds)):]

train_loader = DataLoader(
    dataset,
    num_workers=workers,
    batch_size=batch_size,
    sampler=SubsetRandomSampler(train_inds)
)
val_loader = DataLoader(
    dataset,
    num_workers=workers,
    batch_size=batch_size,
    sampler=SubsetRandomSampler(val_inds)
)

#### Define loss and evaluation functions

In [70]:
loss_fn = torch.nn.CrossEntropyLoss()
metrics = {
    'fps': training.BatchTimer(),
    'acc': training.accuracy
}

#### Train model

In [71]:
writer = SummaryWriter()
writer.iteration, writer.interval = 0, 10

print('\n\nInitial')
print('-' * 10)
resnet.eval()
training.pass_epoch(
    resnet, loss_fn, val_loader,
    batch_metrics=metrics, show_running=True, device=device,
    writer=writer
)

for epoch in range(epochs):
    print('\nEpoch {}/{}'.format(epoch + 1, epochs))
    print('-' * 10)

    resnet.train()
    training.pass_epoch(
        resnet, loss_fn, train_loader, optimizer, scheduler,
        batch_metrics=metrics, show_running=True, device=device,
        writer=writer
    )

    resnet.eval()
    training.pass_epoch(
        resnet, loss_fn, val_loader,
        batch_metrics=metrics, show_running=True, device=device,
        writer=writer
    )

writer.close()



Initial
----------
Valid |     1/1    | loss:    2.5640 | fps:    6.7321 | acc:    0.0000   

Epoch 1/8
----------
Train |     3/3    | loss:    1.2646 | fps:   12.8763 | acc:    0.5982   
Valid |     1/1    | loss:   19.6287 | fps:   37.6981 | acc:    0.4091   

Epoch 2/8
----------
Train |     3/3    | loss:    0.4519 | fps:   13.3845 | acc:    0.8795   
Valid |     1/1    | loss:   22.5744 | fps:   38.2497 | acc:    0.4545   

Epoch 3/8
----------
Train |     3/3    | loss:    0.3244 | fps:   12.7516 | acc:    0.8899   
Valid |     1/1    | loss:   18.2057 | fps:   39.2663 | acc:    0.3182   

Epoch 4/8
----------
Train |     3/3    | loss:    0.1790 | fps:   13.1053 | acc:    0.9216   
Valid |     1/1    | loss:    2.5365 | fps:   36.5574 | acc:    0.6364   

Epoch 5/8
----------
Train |     3/3    | loss:    0.2388 | fps:   12.3262 | acc:    0.9474   
Valid |     1/1    | loss:    3.4522 | fps:   37.1260 | acc:    0.5000   

Epoch 6/8
----------
Train |     3/3    | loss:    0.2